# Creo un nuovo dataset 


---
## Analisi dei commenti di un ristorante

### Da Gianni - Gorizia, FVG

URL di riferimento: https://www.tripadvisor.com/Restaurant_Review-g635871-d1006773-Reviews-Trattoria_Da_Gianni-Gorizia_Province_of_Gorizia_Friuli_Venezia_Giulia.html

Home --> Ristoranti --> Gorizia --> Best_ranking--> Trattoria da Gianni

In [1]:
# IMPORT NECESSARI
import requests
from bs4 import BeautifulSoup

print("Imported")

Imported


In [2]:
page_URL = "https://www.tripadvisor.it/Restaurant_Review-g635871-d1006773-Reviews-Trattoria_Da_Gianni-Gorizia_Province_of_Gorizia_Friuli_Venezia_Giulia.html"

data = requests.get(page_URL)
clear_data = BeautifulSoup(data.content, "lxml")

print("Done")

Done


---
### STRUTTURA HTML DEL SITO 

#### REVIEW
`<p class="partial_entry"> qui c'è la review </p>`

#### N° TOTALE DI REVIEWS
`<a data-page-number="25" ...>
    25
 </a>`
 
#### NAMES
`<span class="expand_inline scrname" …>AlexandraCapeTown</span>`

#### RATINGS
`<span class="ui_bubble_rating bubble_30"></span>`

---
### PRIMO SCRAPING (sulla prima pagina)

Ottengo i nomi, i commenti, le review e da dove è stato scritto il commento per quanto rigurda la prima pagina. Ciò è necessario, in quanto, dalla seconda in poi l'URL cambia, permettendoci di iterare.

In [3]:
# OTTENGO UNA LISTA DI DIV CHE CONTENGONO NOME, RECENSIONE E VOTO

all_reviews = []
all_names = []
all_ratings = []
all_mobile = []

review_container = clear_data.findAll("div", attrs={"class":"review-container"})

for container in review_container:
  
    ### REVIEW ###
    review = container.find('p', attrs={'class': 'partial_entry'})
    if review:
        all_reviews.append(review.text)
        
    ### NAME ###
    name = container.find('span', attrs={'class': ('expand_inline', 'scrname')})
    if name:
      all_names.append(name.text)
    else:
      all_names.append('NO_NAME')
    
    ### RATING ###
    rating = container.find('span', attrs={'class': 'ui_bubble_rating'})
    all_ratings.append(rating['class'][1][7:8])
    
    ### MOBILE ###
    mobile = container.find('span', attrs={'class': 'viaMobile'})
    if mobile:
      all_mobile.append('Mobile')
    else:
      all_mobile.append('PC')
    
    
################################################################################################################################
        
print("Executed")
print('Nomi: ', len(all_names))
print('Commenti: ', len(all_reviews))
print('Voti: ', len(all_ratings))
print('Scritto da: ', len(all_ratings))

Executed
Nomi:  10
Commenti:  10
Voti:  10
Scritto da:  10


### SECONDO SCRAPING (dalla seconda pagina in poi)

Notiamo dall'URL che è possibile iterare sulle pagine, ottenendo i dati!

In [4]:
from tqdm import tqdm

URL_COMPLETO = 'https://www.tripadvisor.it/Restaurant_Review-g635871-d1006773-Reviews-Trattoria_Da_Gianni-Gorizia_Province_of_Gorizia_Friuli_Venezia_Giulia.html'

URL_p1 = 'https://www.tripadvisor.it/Restaurant_Review-g635871-d1006773-Reviews-or'
URL_p2 = '0-Trattoria_Da_Gianni-Gorizia_Province_of_Gorizia_Friuli_Venezia_Giulia.html'
    
# OTTENGO I COMMENTI, NOMI E VOTI DALLA SECONDA PAGINA IN POI

total_pages = int(clear_data.select("a.pageNum.last.taLnk")[0].text) # 25 come intero

# per iterare dobbiamo partire dalla seconda pagina dei commenti, così nell'URL compaiono le cifre che ci servono
for r in tqdm(range(1,total_pages)):

    # OTTENGO L'URL DELLA PAGINA
    URL_temp = URL_p1 + str(r) + URL_p2

    # OTTENGO I DATI E CI APPLICO BEATIFULSOUP
    data = requests.get(URL_temp)
    clear_data = BeautifulSoup(data.content, "lxml")
    

    review_container = clear_data.findAll("div", attrs={"class":"review-container"})

    for container in review_container:
  
      ### REVIEW ###
      review = container.find('p', attrs={'class': 'partial_entry'})
      if review:
          all_reviews.append(review.text)

      ### NAME ###
      name = container.find('span', attrs={'class': ('expand_inline', 'scrname')})
      if name:
        all_names.append(name.text)
      else:
        all_names.append('NO_NAME')

      ### RATING ###
      rating = container.find('span', attrs={'class': 'ui_bubble_rating'})
      all_ratings.append(rating['class'][1][7:8])

      ### MOBILE ###
      mobile = container.find('span', attrs={'class': 'viaMobile'})
      if mobile:
        all_mobile.append('Mobile')
      else:
        all_mobile.append('PC')
    
        
################################################################################################################################
        
print("Executed")
print('total_pages: ', total_pages)
print('Nomi: ', len(all_names))
print('Commenti: ', len(all_reviews))
print('Voti: ', len(all_ratings))
print('Scritto da: ', len(all_mobile))

100%|██████████| 179/179 [03:10<00:00,  1.55s/it]

Executed
total_pages:  180
Nomi:  1798
Commenti:  1798
Voti:  1798
Scritto da:  1798


In [5]:
# display(all_reviews)

In [6]:
# print(all_names)

In [7]:
#print(all_ratings, '\n')
#print(len(all_ratings))

In [8]:
#display(all_mobile)

#### Problema recensioni troppo lunghe
Alla fine di alcune recensioni troviamo un '...Più', che non possiamo espandere. Data la mole di dati, decidiamo dunque di non considerarle.

In [9]:
all_reviews_filtered = []
counter_reviews_scartate = 0;
counter_reviews_da_usare = 0;

for review in all_reviews:
    if '...Più' in review:
        all_reviews_filtered.append('RECENSIONE_TROPPO_LUNGA')
        counter_reviews_scartate+=1;
    else:
        all_reviews_filtered.append(review)
        counter_reviews_da_usare+=1;
        
        
print(
    'Totale recensioni:', counter_reviews_scartate + counter_reviews_da_usare, '\n',
    'Recensioni analizzabili:', counter_reviews_da_usare, '\n',
    'Recensioni scartate:', counter_reviews_scartate, '\n')

display(all_reviews_filtered[:10])

Totale recensioni: 1798 
 Recensioni analizzabili: 689 
 Recensioni scartate: 1109 



['RECENSIONE_TROPPO_LUNGA',
 'Come già riportato da tanti altri utenti,non si può che parlare soprattutto dell’abbondanza delle porzioni che contraddistingue questo locale... Il cibo è buono,il servizio è veloce ed efficiente... Da provare assolutamente se ci si trova nei pressi di Gorizia.',
 'RECENSIONE_TROPPO_LUNGA',
 'Trattoria fantastica, porzioni abbondanti e cibo buonissimo! Consigliatissimo! Dovete andarci assolutamente!',
 'Non conoscevo questo locale, mi ha portato mia figlia, molto familiare e casalingo buon cibo ma portate esageratamente abbondanti, perfetto per i mangioni :))))',
 'RECENSIONE_TROPPO_LUNGA',
 'RECENSIONE_TROPPO_LUNGA',
 'Non ho mai trovato in vita mia un ristorante con delle porzioni simili.\nHo ordinato un piatto di cevapcici e me ne hanno portato 40. Mia nipote che voleva una braciola di maiale gliene hanno portate 5. Almeno una volta bisogna andarci',
 'RECENSIONE_TROPPO_LUNGA',
 "Forte per le quantità o per cene goliardiche con gli amici ma la qualità è

---
### Analisi dei nomi
Cerchiamo di capire se chi ha commentato è un uomo (M) o una donna (F).

#### Elimino spazi e numeri
Ho bisogno di tenere solo le informazioni riguardanti il nome, elimino il resto.

In [10]:
import re

all_names_filtered = []

regex = re.compile('[^a-zA-Z]')

for name in all_names:
    
    if ' ' in name:                           # se c'è uno spazio nel nome
        space = name.index(' ')               # ne trovo l'indice
        name = regex.sub('', name[:space])    # effettuo un controllo su tutto ciò che non è alfabeto fino a quell'indice
        all_names_filtered.append(name)       # e aggiungo la stringa
        
    else:
        name = regex.sub('', name)            # in caso contrario effettuo un controllo su tutto ciò che non è alfabeto
        all_names_filtered.append(name)       # e aggiungo la stringa
        
print(all_names, '\n')
print(all_names_filtered)

['NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'janinci j', 'ChefDonnie', 'Giuliaso', 'nicoletta494', 'Milva319', 'LaEly3', 'annabD2781JQ', 'Sonia S', 'Trip_che_fame', 'maxino769', 'ChiaraUdine89', 'simonebisaggio', 'Tiziana C', 'AntoninoLu', 'flavjo', 'Sara C', '765lorenzos', 'micR73', 'U6846WGpaolo', 'AntoninoF89', 'piero031958', 'Italikus', 'Daniele P', 'deborad760', 'Sibilla P', 'Irene D', '201inesp', 'duccioc264', 'morands81', 'renzo g', 'Sandro G', 'riccardoi435', 'Fabiola t', 'J9229ESmarcov', 'Giampiero C', 'Florindo F', 'tinetta97', '416des', 'matand', 'Laura-e-oshy', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', '225vincem', 'Aghi46', 'Tommaso B', 'Llfg80', '443matijak', 'simonebC2577KW', 'Vanni73', 'Massimo P', 'LucaBelvedere85', 'maromans91', 'ChefDonnie', 'LoveBrain', 'alessio8299', '77claudiabor', 'Luca D', 'La_talpa888', 'MrBirra', 'emanuele09098

#### Utilizzo gender_guesser
Infine, posso utilizzare la libreria gender_guesser.

NB: Genderize ti permette di analizzare 1000 nomi al giorno.

In [11]:
!pip install gender_guesser

In [12]:
import gender_guesser.detector as gender
d = gender.Detector()

names_with_gender = []

for name in all_names_filtered:
    temp_gender = d.get_gender(name)
    names_with_gender.append(temp_gender)
  
print(names_with_gender)

['unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'female', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'female', 'unknown', 'male', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'male', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'male', 'unknown', 'female', 'unknown', 'unknown', 'mostly_male', 'male', 'unknown', 'female', 'unknown', '

In [13]:
all_genders = []
i = 0

for guess in names_with_gender:
    if guess == 'male' or guess == 'mostly_male':
        all_genders.append('M')
    elif guess == 'female' or guess == 'mostly_female':
        all_genders.append('F')
    else:
        all_genders.append('UNKNOWN')

    
print(all_genders)

['UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'F', 'UNKNOWN', 'UNKNOWN', 'F', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'F', 'UNKNOWN', 'UNKNOWN', 'F', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'F', 'UNKNOWN', 'F', 'F', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M', 'UNKNOWN', 'F', 'UNKNOWN', 'M', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M', 'UNKNOWN', 'UNKNOWN', 'M', 'UNKNOWN', 'F', 'UNKNOWN', 'UNKNOWN', 'M', 'M', 'UNKNOWN', 'F', 'UNKNOWN', 'UNKNOWN', 'F', 'UNKNOWN', 'UNKNOWN', 'F', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNK

---
## Salviamo ora i dati ottenuti in una tabella Excel
Così non dovremo ogni volta rieseguire tutto il codice



### Controlliamo che ci siano lo stesso numero di dati
Potremo così creare una tabella sfruttando Pandas

In [14]:
print('Nomi: ', len(all_names_filtered))
print('Commenti: ', len(all_reviews_filtered))
print('Voti: ', len(all_ratings))
print('Mobile: ', len(all_mobile))

Nomi:  1798
Commenti:  1798
Voti:  1798
Mobile:  1798


### Eliminiamo le recensioni troppo lunghe e quelle di cui non sappiamo il genere del commentatore
Creo una lista di liste [ ['NOME', 'GENERE', 'RECENSIONE', 'VOTO'], ['NOME', 'GENERE', 'RECENSIONE', 'VOTO'], ... ]

In [15]:
complete_list = []

i = len(all_names_filtered)

for indice in range(i):
  if all_genders[indice] != 'UNKNOWN' and all_reviews_filtered[indice] != 'RECENSIONE_TROPPO_LUNGA':
    temp = [all_names_filtered[indice], all_genders[indice], all_reviews_filtered[indice], all_ratings[indice], all_mobile[indice]]
    complete_list.append(temp)
    
print('DATI ANALIZZABILI: ', len(complete_list))
display(complete_list[:5])


DATI ANALIZZABILI:  146


[['Milva',
  'F',
  "E' un locale centrale a Gorizia e conosciutissimo per le dimensioni enormi dei piatti che serve. Si tratta di cucina tipica.",
  '4',
  'PC'],
 ['Sonia',
  'F',
  'Tutto moltooooo abbondante, noi abbiamo preso delle mozzarelle in carrozza supersquisite, una portata di tagliata di Manzo,delle salsicciette e non poteva mancare l enorme lubianska. Tutto super buono ritorneremo sicuramente. Un consiglio prenotate anticipatamente.',
  '5',
  'Mobile'],
 ['Sibilla',
  'F',
  "Cucina tipica, ottima qualità e porzioni che sconvolgono i forestieri! Consiglio l'asporto per tentare di finire, con tutta calma, le pietanze. NB: solo chi non ha mai provato la lubianska di Gianni... Osa chiedere anche le patatine fritte. Impossibile mangiare entrambe le portate!!!",
  '5',
  'Mobile'],
 ['Irene',
  'F',
  'Tutto buono e abbondante!!! Complimenti per i primi e la Lubjanska... siamo venuti per questo e non siamo rimasti delusi! Avrei voluto assaggiare anche i dolci, ma.... proprio 

### Creiamo la tabella

In [16]:
# !pip install pandas
# !pip install xlsxwriter
import pandas as pd

In [17]:
# Create a Pandas dataframe from the data.
df = pd.DataFrame({'Nomi': [elemento[0] for elemento in complete_list], 
                   'Genere': [elemento[1] for elemento in complete_list], 
                   'Recensioni': [elemento[2] for elemento in complete_list],
                   'Voti': [elemento[3] for elemento in complete_list],
                   'Scritta da': [elemento[4] for elemento in complete_list]})

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('RISTORANTE_16.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df.to_excel(writer, sheet_name='Sheet1')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

print('done')

done


In [18]:
partial_dataframe = pd.read_excel("RISTORANTE_16.xlsx")

display(partial_dataframe)

,Nomi,Genere,Recensioni,Voti,Scritta da
0,Milva,F,E' un locale centrale a Gorizia e conosciutiss...,4,PC
1,Sonia,F,"Tutto moltooooo abbondante, noi abbiamo preso ...",5,Mobile
2,Sibilla,F,"Cucina tipica, ottima qualità e porzioni che s...",5,Mobile
3,Irene,F,Tutto buono e abbondante!!! Complimenti per i ...,4,Mobile
4,Fabiola,F,"Cibo buono ed abbondante, personale cordiale. ...",5,Mobile
5,Tommaso,M,Nulla da aggiungere su quella che è una vera e...,5,Mobile
6,Vanni,M,Cu sono andato con amici ed è stata una bella ...,4,Mobile
7,Luca,M,"Il locale è semplice, non propriamente raffina...",5,PC
8,Il,M,"Molto buono direi. Consiglio Lubjanska, patate...",4,Mobile
9,Angus,M,"A Gorizia oramai da anni, quando si parla di t...",5,Mobile
